In [3]:
import pandas as pd
from sklearn.cluster import KMeans
import folium


In [11]:
# 📥 Read CSV file
file_path = "/content/drive/MyDrive/Colab Notebooks/mini/sample_disaster_data.csv"  # Change this if you're uploading to Colab
df = pd.read_csv(file_path)
df


,Location,Latitude,Longitude,Disaster_Type,Casualties,Infrastructure_Damage
0,City A,28.6,77.20,Flood,100,3
1,City B,28.7,77.30,Landslide,50,2
2,City C,28.5,77.10,Flood,70,4
3,City D,28.9,77.40,Earthquake,200,5
4,City E,28.4,77.05,Storm,30,1
5,City F,28.8,77.35,Flood,90,3


In [12]:
# 🧠 Features for clustering
features = df[["Latitude", "Longitude", "Casualties", "Infrastructure_Damage"]]


In [13]:
# 🤖 Apply K-Means clustering
kmeans = KMeans(n_clusters=3, random_state=0)
df["Cluster"] = kmeans.fit_predict(features)


In [14]:
# 🌍 Create map centered on the mean location
center_lat, center_lon = df["Latitude"].mean(), df["Longitude"].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=10)



In [15]:
# 🎨 Cluster color palette
colors = ['red', 'green', 'blue']

In [16]:
# 🗺️ Plot each point
for _, row in df.iterrows():
    folium.CircleMarker(
        location=[row["Latitude"], row["Longitude"]],
        radius=10 + row["Casualties"] / 30,
        color=colors[row["Cluster"]],
        fill=True,
        fill_opacity=0.6,
        tooltip=f"{row['Location']} | {row['Disaster_Type']} | Cluster {row['Cluster']}"
    ).add_to(m)

# Show map
m


In [17]:
def recommend_resources(row):
    resources = []

    # Casualty-based ambulance allocation
    if row["Casualties"] >= 100:
        resources.append("5 ambulances")
    elif row["Casualties"] >= 50:
        resources.append("3 ambulances")
    else:
        resources.append("1 ambulance")

    # Infrastructure damage → Shelters
    if row["Infrastructure_Damage"] >= 4:
        resources.append("2 shelters")

    # Disaster-specific resources
    if row["Disaster_Type"].lower() == "flood":
        resources.append("2 boats")
    elif row["Disaster_Type"].lower() == "landslide":
        resources.append("1 excavator")

    return ", ".join(resources)

# Apply to each row
df["Recommended_Resources"] = df.apply(recommend_resources, axis=1)

# Show result
df[["Location", "Disaster_Type", "Casualties", "Infrastructure_Damage", "Recommended_Resources"]]


,Location,Disaster_Type,Casualties,Infrastructure_Damage,Recommended_Resources
0,City A,Flood,100,3,"5 ambulances, 2 boats"
1,City B,Landslide,50,2,"3 ambulances, 1 excavator"
2,City C,Flood,70,4,"3 ambulances, 2 shelters, 2 boats"
3,City D,Earthquake,200,5,"5 ambulances, 2 shelters"
4,City E,Storm,30,1,1 ambulance
5,City F,Flood,90,3,"3 ambulances, 2 boats"


In [18]:
# 🔧 Current available inventory
resource_inventory = {
    "ambulances": 10,
    "boats": 5,
    "shelters": 6
}


In [19]:
resource_stations = [
    {"name": "Base 1", "lat": 28.7, "lon": 77.1},
    {"name": "Base 2", "lat": 28.5, "lon": 77.3},
    {"name": "Base 3", "lat": 28.6, "lon": 77.25},
]


In [20]:
from geopy.distance import geodesic

def find_closest_base(disaster_lat, disaster_lon):
    min_dist = float('inf')
    best_base = None
    for base in resource_stations:
        dist = geodesic((disaster_lat, disaster_lon), (base["lat"], base["lon"])).km
        if dist < min_dist:
            min_dist = dist
            best_base = base["name"]
    return best_base, round(min_dist, 2)


In [21]:
allocation_log = []

# Extract resources needed from recommendations
for idx, row in df.iterrows():
    recommended = row["Recommended_Resources"].split(", ")
    location = row["Location"]
    lat, lon = row["Latitude"], row["Longitude"]

    for item in recommended:
        count, res_type = item.split(" ", 1)
        count = int(count)
        res_key = res_type.rstrip("s").lower() + "s"  # normalize to match inventory dict

        # Check inventory
        if resource_inventory.get(res_key, 0) >= count:
            resource_inventory[res_key] -= count
            base, dist = find_closest_base(lat, lon)
            allocation_log.append(f"✅ Assigned {count} {res_type} to {location} from {base} ({dist} km)")
        else:
            allocation_log.append(f"❌ NOT ENOUGH {res_type.upper()} for {location} (Needed: {count}, Available: {resource_inventory.get(res_key, 0)})")


In [22]:
print("📦 Resource Allocation Plan:\n")
for line in allocation_log:
    print(line)

print("\n📊 Remaining Inventory:", resource_inventory)


📦 Resource Allocation Plan:

✅ Assigned 5 ambulances to City A from Base 3 (4.89 km)
✅ Assigned 2 boats to City A from Base 3 (4.89 km)
✅ Assigned 3 ambulances to City B from Base 3 (12.11 km)
❌ NOT ENOUGH EXCAVATOR for City B (Needed: 1, Available: 0)
❌ NOT ENOUGH AMBULANCES for City C (Needed: 3, Available: 2)
✅ Assigned 2 shelters to City C from Base 3 (18.39 km)
✅ Assigned 2 boats to City C from Base 3 (18.39 km)
❌ NOT ENOUGH AMBULANCES for City D (Needed: 5, Available: 2)
✅ Assigned 2 shelters to City D from Base 3 (36.33 km)
✅ Assigned 1 ambulance to City E from Base 2 (26.88 km)
❌ NOT ENOUGH AMBULANCES for City F (Needed: 3, Available: 1)
❌ NOT ENOUGH BOATS for City F (Needed: 2, Available: 1)

📊 Remaining Inventory: {'ambulances': 1, 'boats': 1, 'shelters': 2}
